The Router Chain in [LangChain](https://www.langchain.com/) is used with [Anyscale Endpoints](https://console.endpoints.anyscale.com/) 
to write a Mixed-expert Q & A AI system. Each "subject" such as Math, Physics, Astronomy, Cosmology, Football, or Computer Science has a specific LangChain bound to specific Llama 2 type of model. 

Anyscale Endpoints support these open-source LLM models today:
 * meta-llama/Llama-2-7b-chat-hf
 * meta-llama/Llama-2-13b-chat-hf
 * meta-llama/Llama-2-70b-chat-hf
 * codellama/CodeLlama-34b-Instruct-hf
 
<img src="anyscale_endpoints.png" height="35%" width="%75">
 
You can think of these specific subject matter experts as LLMs that have been fine-tuned models with specific tasks or suitable to only answers questions related to subject area expertise.

<img src="router_chain.png">

This particular example is an extension of Router Chain disscussed 
in the [Deeplearning.ai and LangChain course](https://learn.deeplearning.ai/langchain/lesson/4/chains) by Andrew Ng and Harrison Chase, modified and extended here to work with [Anyscale Endpoints](https://console.endpoints.anyscale.com/)

In [ ]:
import warnings
import os

import openai
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv, find_dotenv
from router_prompts import (PromptInfo, MULTI_PROMPT_ROUTER_TEMPLATE)

import gradio as gr

In [ ]:
_ = load_dotenv(find_dotenv()) # read local .env file
MODEL = 'meta-llama/Llama-2-13b-chat-hf'
warnings.filterwarnings('ignore')
openai.api_base = os.getenv("ANYSCALE_API_BASE", os.getenv("OPENAI_API_BASE"))
openai.api_key = os.getenv("ANYSCALE_API_KEY", os.getenv("OPENAI_API_KEY"))

In [ ]:
# Create your default model
llm_default = ChatOpenAI(temperature=0.9, model_name=MODEL, streaming=True)

Build destination chains, with each unique LLMChain
bound to a specify type of Llama 2 model and
the subject matter expert prompt to answer questions. 
For example, a physics question will be associated with a physics
LLMChain and related trained model.

In [ ]:
# dictionary of chains
destination_chains = {}
prompt_infos = PromptInfo().prompt_infos
for p_info in prompt_infos:
     # name of the chain for the subject
    name = p_info["name"] 
    
    # subject template
    prompt_template = p_info["prompt_template"] 
    
    # fine-tuned subject-expert model
    model_t = p_info["model"] 
    llm_t = ChatOpenAI(temperature=0.9, model_name=model_t, streaming=True)
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm_t, prompt=prompt) # create the chain and its bound model
    destination_chains[name] = chain  

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
# default prompt and chain if none matches
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm_default, prompt=default_prompt)

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm_default, router_prompt)

chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
questions = [
             "What happens when two galaxies collide?",
             "Compute a prime number larger than 1 and less than 17",
             "What's the time and space complexity of merge sort compared to quicksort",
             "Which is the best open source LLM model would recommend to use today",
             "When was Armstice treaty signed and where and why",
             "What about that actor Micheal Caine, who loves Gooner footie?"
            ]

In [ ]:
def send_prompt(text: str):
    return chain.run(text)
    

In [19]:
mutli_demo= gr.Interface(fn=send_prompt, inputs="text", outputs="text")
mutli_demo.launch()   

Running on local URL:  http://127.0.0.1:7862

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB

To create a public link, set `share=True` in `launch()`.




> Entering new MultiPromptChain chain...
computer science: {'input': 'Which open source LLM model would you recommend using today?'}
> Finished chain.


> Entering new MultiPromptChain chain...
computer science: {'input': 'Write a Shell script to list files in a directory'}
> Finished chain.


> Entering new MultiPromptChain chain...
history: {'input': 'When was the Armistice treaty signed and where and why'}
> Finished chain.


> Entering new MultiPromptChain chain...
math: {'input': 'What is the equation for a Pythagorean triangle?'}
> Finished chain.


> Entering new MultiPromptChain chain...
football: {'input': 'What about that geezer actor Micheal Caine, who loves Gooner footie?'}
> Finished chain.


> Entering new MultiPromptChain chain...
football: {'input': 'What about that actor Micheal Caine, who loves Gooner footie?'}
> Finished chain.


> Entering new MultiPromptChain chain...
football: {'input': 'Tell me the best Gooner football joke from a bloke from east end'}
> Finish

### Use this for debugging 

In [ ]:
for question in questions:
    print("---" * 5)
    print(f"question: {question}")
    print(chain.run(question))